In [215]:
!pip install selenium
!pip install bs4
!pip install pandas
# brew install chromedriver via terminal before continue

In [190]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup 
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.maximize_window()

In [191]:
def login():
    try:
        driver.get('https://auth.finnomena.com/')
        driver.find_element(By.NAME,'email').send_keys('###EMAIL###')
        driver.find_element(By.CLASS_NAME,'custom-button').click()
        driver.find_element(By.NAME,'current_password').send_keys('###PASSWORD###')
        driver.find_element(By.CLASS_NAME,'custom-button').click()
    except Exception as e:
        print(e)
login()

In [210]:
def getStockDetails(name,isAnnual):
    # stock-growthrate
    sections = ['stock-growthrate','stock-financial-report','stock-financial-ratio','stock-statistics']
    url = 'https://www.finnomena.com/stock/' + name
    driver.get(url)
    headers = []
    quarters = []
    data = []
    toggled = False
    for section in sections:
            obj = {}
            keys = []
            names = []
            headerElements = driver.find_element(By.CSS_SELECTOR,f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.topic-wrapper.user-select-none.float-left.overflow-shadow')
            if len(headers) == 0:
                for div in BeautifulSoup(headerElements.get_attribute('innerHTML'), 'html.parser').find_all('div', {'class': 'topic'}):
                    headers.append(div.get_text())
            if not toggled and isAnnual:
                toggled = True
                e = driver.find_element(By.CLASS_NAME,'toggle')
                webdriver.ActionChains(driver).move_to_element(e).click(e).perform()
            for topic in headerElements.find_elements(By.CLASS_NAME,'topic')[1:]:
                key = BeautifulSoup(topic.get_attribute('id')).get_text()
                name = BeautifulSoup(topic.get_attribute('innerHTML'), 'html.parser').get_text()
                keys.append(key)
                names.append(name)
                
                
            contentElements = driver.find_element(By.CSS_SELECTOR, f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.content-wrapper.user-select-none')
            contentHTML = BeautifulSoup(contentElements.get_attribute('innerHTML'), 'html.parser')
            dataWrapper = contentHTML.find_all('div', {'class': 'data-wrapper'})
            
            if len(quarters) == 0:
                for div in contentHTML.find_all('div', {'class': 'year'}):
                    quarters.append(div.get_text())
            
            for i in range(len(dataWrapper)): 
                values = [data.get_text() for data in dataWrapper[i].find_all('div', {'class': 'data-each'})]
                key = f"{keys[i]} {names[i]}"
                obj[key] = values
                data.append(obj)
    return (headers,quarters,data)

In [202]:
import pandas as pd
def getFinancialDetails(symbol,isAnnual):
    headers,quarterlyOrAnnual,responseData = getStockDetails(name=symbol,isAnnual=isAnnual)
    data = {}
    for d in responseData:
        data = dict(list(data.items()) + list(d.items()))
    data['index'] = quarterlyOrAnnual
    df = pd.DataFrame(data) 
    df = df.iloc[:-1]
    df.set_index('index',inplace = True)
    df = df.transpose()
    pf =  "_annual" if isAnnual else "_quarter"
    df.to_csv(f'./Reports/{symbol}{pf}.csv')
    return df

In [212]:
getFinancialDetails(symbol='PRINC', isAnnual=True)

index,2554,2555,2556,2557,2558,2559,2560,2561,2562,2563
Revenue รายได้รวม,163.83,195.99,296.46,373.77,433.02,363.26,"2,395.97","2,856.50","3,158.87","2,758.05"
RevenueYoY การเติบโตต่อปี (%),-48.77,19.63,51.26,26.08,15.85,-16.11,559.58,19.22,10.59,-12.69
NetProfit กำไรสุทธิ,-91.67,-58.46,-1.19,12.96,17.53,-51.30,-345.10,-173.40,-181.72,-511.69
NetProfitYoY การเติบโตต่อปี (%),,36.23,97.97,,35.27,,-572.70,49.75,-4.80,-181.58
EarningPerShare กำไรต่อหุ้น (EPS),-0.20,-0.13,-0.00,0.01,0.02,-0.04,-0.11,-0.05,-0.06,-0.15
EarningPerShareYoY การเติบโตต่อปี (%),,35.00,98.46,,42.86,,-205.71,49.53,-3.70,-164.29
Asset สินทรัพย์รวม,"1,978.28","1,932.02","2,035.67","2,039.97","2,087.29","9,578.28","13,088.21","14,318.08","14,620.05","15,415.11"
TotalDebt หนี้สินรวม,926.97,939.19,108.33,83.38,101.47,"1,747.60","4,164.11","5,476.46","4,908.82","6,090.19"
Equity ส่วนของผู้ถือหุ้น,"1,051.30",992.84,"1,927.34","1,956.60","1,985.82","7,830.68","8,246.29","8,077.56","8,809.73","8,297.13"
PaidUpCapital มูลค่าหุ้นที่เรียกชำระแล้ว,466.90,466.90,933.80,947.72,957.98,"3,240.09","3,240.09","3,240.09","3,462.34","3,462.34"


In [213]:
getFinancialDetails(symbol='PRINC', isAnnual=False)

index,1Q2554,2Q2554,3Q2554,4Q2554,1Q2555,2Q2555,3Q2555,4Q2555,1Q2556,2Q2556,...,3Q2561,4Q2561,1Q2562,2Q2562,3Q2562,4Q2562,1Q2563,2Q2563,3Q2563,4Q2563
Revenue รายได้รวม,40.53,37.49,44.68,41.14,49.36,45.81,48.83,51.99,54.51,46.91,...,786.51,669.21,712.61,694.23,"1,046.81",705.21,724.26,536.04,691.28,806.47
RevenueQoQ การเติบโตต่อไตรมาส (%),-80.40,-7.50,19.18,-7.93,19.99,-7.19,6.59,6.47,4.85,-13.94,...,17.82,-14.91,6.49,-2.58,50.79,-32.63,2.70,-25.99,28.96,16.66
RevenueYoY การเติบโตต่อปี (%),-37.67,100.19,52.74,-80.10,21.80,22.20,9.29,26.38,10.43,2.40,...,406.61,-64.50,-2.81,4.00,33.10,5.38,1.63,-22.79,-33.96,14.36
NetProfit กำไรสุทธิ,-25.90,-24.52,-18.78,-22.47,-12.84,-15.69,-14.96,-14.97,-7.72,-15.21,...,-12.72,-119.20,-73.57,-101.80,165.14,-171.49,-117.69,-169.12,-137.13,-87.73
NetProfitQoQ การเติบโตต่อไตรมาส (%),,5.36,23.40,-19.68,42.85,-22.13,4.61,-0.03,48.44,-97.02,...,79.72,-837.07,38.28,-38.37,,,31.37,-43.70,18.91,36.02
NetProfitYoY การเติบโตต่อปี (%),-18.04,30.76,30.39,,50.42,36.02,20.32,33.40,39.91,3.06,...,54.26,-5.37,,-62.32,,-43.87,-59.99,-66.14,,48.84
EarningPerShare กำไรต่อหุ้น (EPS),-0.06,-0.05,-0.04,-0.05,-0.03,-0.03,-0.03,-0.04,-0.02,-0.03,...,-0.00,-0.04,-0.02,-0.03,0.05,-0.05,-0.03,-0.05,-0.04,-0.03
EarningPerShareQoQ การเติบโตต่อไตรมาส (%),,16.67,20.00,-25.00,40.00,-3.67,7.07,-38.41,50.00,-50.00,...,80.00,-825.00,37.84,-34.78,,,35.85,-44.12,20.41,33.33
EarningPerShareYoY การเติบโตต่อปี (%),-20.00,28.57,33.33,,50.00,37.80,27.75,20.00,33.33,3.54,...,55.56,-5.71,,-55.00,,-43.24,-47.83,-58.06,,50.94
Asset สินทรัพย์รวม,"2,050.23","2,026.58","2,000.53","1,978.28","1,966.33","1,951.92","1,937.90","1,932.02","1,904.49","1,870.89",...,"13,787.04","14,318.08","14,695.50","14,939.59","14,897.21","14,620.05","14,867.32","15,174.49","15,082.98","15,415.11"
